# Fields for Experts 

Initially, we chose 250 fields manually for experts. Here I will make more sents by code.

In [1]:
import pandas as pd

In [2]:
param_in_data_dir = "/Users/hn/Documents/01_research_data/NASA/parameters/"

## Read all the ~6000 fields chosen previously

In [3]:
evaluation_set = pd.read_csv(param_in_data_dir + "Eshwar_Extensive.csv")
evaluation_set.drop(labels=['Text box Text', 'Question_overall'], axis='columns', inplace=True)
evaluation_set.head(1)

,Question_overall,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude,Question Text,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,1,104119_WSDA_SF_2016,AdamBenton2016_104119_WSDA_SF_2016.png,104119_WSDA_SF_2016_46.83273927_-118.91209151_...,104119_WSDA_SF_2016_46.83273927_-118.91209151_...,46.832739,-118.912092,How would you label this field?,alfalfa hay,center pivot,wsda,26,26.413875,2016/08/17 00:00:00,Adams


In [4]:
wanted_crops = ['bean, dry', 'bean, green', 'buckwheat', 'canola', 'carrot',
                'triticale', 'market crops', 'grass hay', 'mint', 'onion', 
                'barley', 'barley hay', 'oat hay', 'yellow mustard', 'corn seed', 
                'corn, field', 'corn, sweet', 'wheat', 'potato', 'alfalfa seed', 'grass seed',
                'bluegrass seed', 'pea seed', 'pea, dry', 'pea, green']
wanted_crops = sorted(wanted_crops)
len(wanted_crops)

25

In [5]:
# Pick up the crops that Kirti wanted
evaluation_set = evaluation_set[evaluation_set.CropTyp.isin(wanted_crops)]

## Read the first set; 295 fields labeled in the first set

In [6]:
perry_dir = "/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/"

In [7]:
csv_fields = pd.read_csv(perry_dir + "set_1_experts_stats_extended_sortOpinionCrop.csv")

In [8]:
for a_crop in wanted_crops:
    a_df = evaluation_set[evaluation_set.CropTyp == a_crop]
    print('There are [{field_count}] crops of type [{curr_crop}].'.format(field_count=a_df.shape[0], 
                                                                          curr_crop=a_crop))

There are [51] crops of type [alfalfa seed].
There are [17] crops of type [barley].
There are [42] crops of type [barley hay].
There are [79] crops of type [bean, dry].
There are [48] crops of type [bean, green].
There are [50] crops of type [bluegrass seed].
There are [50] crops of type [buckwheat].
There are [27] crops of type [canola].
There are [50] crops of type [carrot].
There are [50] crops of type [corn seed].
There are [415] crops of type [corn, field].
There are [89] crops of type [corn, sweet].
There are [132] crops of type [grass hay].
There are [43] crops of type [grass seed].
There are [50] crops of type [market crops].
There are [50] crops of type [mint].
There are [18] crops of type [oat hay].
There are [56] crops of type [onion].
There are [2] crops of type [pea seed].
There are [32] crops of type [pea, dry].
There are [50] crops of type [pea, green].
There are [215] crops of type [potato].
There are [22] crops of type [triticale].
There are [258] crops of type [wheat]

In [10]:
evaluation_set.head(2)

,Question_overall,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude,Question Text,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
483,484,52638_WSDA_SF_2016,AdamBenton2016_52638_WSDA_SF_2016.png,52638_WSDA_SF_2016_46.99338282_-118.97934807_c...,52638_WSDA_SF_2016_46.99338282_-118.97934807_T...,46.993383,-118.979348,How would you label this field?,alfalfa seed,rill,wsda,37,37.102631,2016/08/24 00:00:00,Adams
484,485,148920_WSDA_SF_2015,Walla2015_148920_WSDA_SF_2015.png,148920_WSDA_SF_2015_46.03609783_-118.72012153_...,148920_WSDA_SF_2015_46.03609783_-118.72012153_...,46.036098,-118.720122,How would you label this field?,alfalfa seed,sprinkler,wsda,2,2.197327,2015/06/04 00:00:00,Walla Walla


In [9]:
min_no_fields = 10

for set_count in range(2, 5):
    curr_set = pd.DataFrame()
    for a_crop in wanted_crops:
        a_df = evaluation_set[evaluation_set.CropTyp == a_crop].copy()
        row = min(min_no_fields, a_df.shape[0])
        a_df = a_df.iloc[0:row, ].copy()
        curr_set = pd.concat([curr_set, a_df])
    
    out_name = perry_dir + "set_" + str(set_count) + ".csv"
    curr_set.to_csv(out_name, index = False)
    
    # Get rid of fields that are already chosen
    evaluation_set = evaluation_set[~(evaluation_set.ID.isin(list(curr_set.ID)))].copy()

In [ ]:
# set_2 = pd.read_csv(perry_dir + "set_2.csv")
# set_3 = pd.read_csv(perry_dir + "set_3.csv")
# set_4 = pd.read_csv(perry_dir + "set_4.csv")

In [12]:
del(curr_set, set_count)

### Write the sets in excel format

In [16]:
no_questions = 50

writer_limited = pd.ExcelWriter(perry_dir + 'set_2_to_4.xlsx', engine='xlsxwriter') 

for set_count in range(2, 5):
    curr_set = pd.read_csv(perry_dir + "set_" + str(set_count) + ".csv")
    curr_set.sort_values(by=['CropTyp', 'county', "ID"], inplace=True)

    if curr_set.shape[0] % no_questions != 0:
        no_dfs = curr_set.shape[0] // no_questions + 1
    else:
        no_dfs = curr_set.shape[0] // no_questions

    for ii in range(no_dfs):
        curr_sheet = curr_set.loc[(ii*no_questions): ((ii+1) * no_questions) - 1, ]
        curr_sheet.reset_index(drop=True, inplace=True)
        curr_sheet.to_excel(writer_limited, sheet_name= "set_" + str(set_count) + "_form_" + str(ii+1), index=False)

writer_limited.save()

In [30]:
len(evaluation_set.CropTyp.unique())

19